In [1]:
from create_data import create_csv

norm_sim = "/disk/moose/lhcb/djdt/Lb2L1520mueTuples/MCNorm/2016MD/halfSampleFeb22/job246-CombDVntuple-MCNorm-15144059-S28r2Restrip-firstHalf-2016MD-pKmumu-PF__PreselectedV1.root"
norm_real = "/disk/moose/lhcb/djdt/Lb2L1520mueTuples/realDataNorm/2016MD/halfSampleFeb22/job228-CombDVntuple-collision-firstHalf-2016MD-pKmumu-PF__PreselectedV1.root"

rare_sim = "/disk/moose/lhcb/djdt/Lb2L1520mueTuples/MC/2016MD/fullSampleOct2021/job207-CombDVntuple-15314000-MC2016MD_Full-pKmue-MC.root"
rare_real = "/disk/moose/lhcb/djdt/Lb2L1520mueTuples/realData/2016MD/halfSampleOct2021/blindedTriggeredL1520Selec-collision-firstHalf2016MD-pKmue_Fullv9.root"

In [3]:
kwargs = {
    'ver': '9.0.4',
    'sim_path': rare_sim,
    'actual_path': rare_real,
    'decay_tree_name': ':DTT1520me/DecayTree',
    'preselect': True,
    'preselect_path': 'preselection.txt',
    'equalise_event_numbers': False,
    'dropnan': False,
    'restrict_mass': ([4500, 5200], [5800, 6500]),
    'train_fraction': 0.6,
    'val_fraction': 0.2,
    'test_fraction': 0.2,
    'isNormalisation': False,
    'request': 'request.txt',
    'random_seed': 0
}

create_csv(kwargs)

INFO: Reading in data from simulated tree
INFO: Evaluating pre-selection for simulated data
INFO: Currently there are 89269 events
INFO: Preselection applied without error
INFO: Currently there are 18194 events

INFO: Simulated data manipulation complete


/disk/homedisk/home/user211/project/create_data.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf['category'] = np.where(sdf['Lb_BKGCAT'].isin([10, 50]), 1, 0)
/disk/homedisk/home/user211/project/create_data.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf['IsSimulated'] = True


INFO: Evaluating pre-selection for real data
INFO: Currently there are 133541 events
INFO: Preselection applied without error
INFO: Currently there are 31783 events

INFO: Restricting mass sidebands
INFO: Currently there are 31783 events
INFO: There are 11939 events less than the lower mass threshold
INFO: There are 1059 events above the higher mass threshold
INFO: There are 0 events between threshold values
INFO: Mass restriction complete
INFO: Currently there are 18785 events


/disk/homedisk/home/user211/project/create_data.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdf['IsSimulated'] = False
/disk/homedisk/home/user211/project/create_data.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdf['category'] = 0
/cvmfs/lhcbdev.cern.ch/conda/envs/default/2021-09-07_04-06/linux-64/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Output to CSV: data_files/9.0.4/all.csv
Generating metadata file...
INFO: Metadata file generated at outputs/csv_metadata/9.0.4.txt
